In [8]:
import os
import sys
sys.path.append("..")
import numpy as np
import torch
from utils.matching import match_reconstruction_ground_truth
import pandas as pd
from utils import Timer, post_process_continuous
from attacks import train_and_attack_fed_avg
from models import FullyConnected
import matplotlib.pyplot as plt
import argparse
import pickle
from attacks import calculate_random_baseline
from datasets.base_dataset import BaseDataset
torch.set_printoptions(sci_mode=False)
from scipy.optimize import linear_sum_assignment
import pickle

# from scipy.optimize import linear_sum_assignment
# from .eval_metrics import categorical_accuracy_continuous_tolerance_score
# import numpy as np

In [9]:
os.getcwd()

'/home/chiragpandav/Downloads/tableak_FT/updated_code/tableak_adult_gen/result_analysis'

In [10]:
from utils import categorical_accuracy_continuous_tolerance_score

# from .eval_metrics import categorical_accuracy_continuous_tolerance_score


In [11]:


def match_reconstruction_ground_truth(target_batch, reconstructed_batch, tolerance_map, return_indices=False,
                                      match_based_on='all'):
    """
    For a reconstructed batch of which we do not know the order of datapoints reconstructed, as the loss and hence the
    gradient of the loss is permutation invariant with respect to the input batch, this function calculates the optimal
    reordering i.e. matching to the ground truth batch to get the minimal reconstruction error. It uses the
    reconstruction score 'categorical_accuracy_continuous_tolerance_score'.

    :param target_batch: (np.ndarray) The target batch in mixed representation.
    :param reconstructed_batch: (np.ndarray) The reconstructed batch in mixed representation.
    :param tolerance_map: (list) The tolerance map required to calculate the reconstruction score.
    :param return_indices: (bool) Trigger to return the matching index map as well.
    :param match_based_on: (str) Select based on which feature type to match. Available are 'all', 'cat', 'cont'.
    :return: reordered_reconstructed_batch (np.ndarray), batch_cost_all (np.ndarray), batch_cost_cat (np.ndarray),
        batch_cost_cont (np.ndarray): The correctly reordered reconstructed batch, the minimal cost vectors of all
        feature costs, only categorical feature costs, only continuous feature costs.
    """
    assert match_based_on in ['all', 'cat', 'cont'], 'Please select a valid matching ground from all, cat, cont'

    if len(target_batch.shape) != 2 or len(reconstructed_batch.shape) != 2:
        target_batch = np.reshape(target_batch, (-1, len(tolerance_map)))
        reconstructed_batch = np.reshape(reconstructed_batch, (-1, len(tolerance_map)))

    batch_size = target_batch.shape[0]

    # create the cost matrix for matching the reconstruction with the true data to calculate the score
    cost_matrix_all, cost_matrix_cat, cost_matrix_cont = [np.zeros((batch_size, batch_size)) for _ in range(3)]
    for k, target_data_point in enumerate(target_batch):
        for l, recon_data_point in enumerate(reconstructed_batch):
            cost_all, cost_cat, cost_cont = categorical_accuracy_continuous_tolerance_score(
                target_data_point, recon_data_point, tolerance_map, True)
            cost_matrix_all[k, l], cost_matrix_cat[k, l], cost_matrix_cont[k, l] = cost_all, cost_cat, cost_cont

    # perform the Hungarian algorithm to match the reconstruction
    if match_based_on == 'all':
        row_ind, col_ind = linear_sum_assignment(cost_matrix_all)
    elif match_based_on == 'cat':
        row_ind, col_ind = linear_sum_assignment(cost_matrix_cat)
    else:
        row_ind, col_ind = linear_sum_assignment(cost_matrix_cont)

    # create the "real" cost vectors and reorder the reconstructed batch according to the matching
    reordered_reconstructed_batch = reconstructed_batch[col_ind]
    batch_cost_all, batch_cost_cat, batch_cost_cont = cost_matrix_all[row_ind, col_ind], \
                                                      cost_matrix_cat[row_ind, col_ind], \
                                                      cost_matrix_cont[row_ind, col_ind]

    if return_indices:
        return reordered_reconstructed_batch, batch_cost_all, batch_cost_cat, batch_cost_cont, row_ind, col_ind
    else:
        return reordered_reconstructed_batch, batch_cost_all, batch_cost_cat, batch_cost_cont


In [12]:
!pip install numpy==1.23.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.8 -m pip install --upgrade pip


In [13]:
import numpy as np
import pickle
np.__version__

'1.22.4'

In [14]:
recn_gt = f'../clients_data/reconstr_and_GT/inversion/normal/reconstructions_ground_truths_0.pkl'
dataset = f'../clients_data/reconstr_and_GT/inversion/normal/dataset_0.pkl'
tolerance_map = f'../clients_data/reconstr_and_GT/inversion/normal/tolerance_map_0.pkl'

with open(tolerance_map, 'rb') as file:
    tolerance_map = pickle.load(file)

with open(recn_gt, 'rb') as file:
    recn_gt = pickle.load(file)

with open(dataset, 'rb') as file:
    dataset = pickle.load(file)

reconstructions = recn_gt['reconstructions']
ground_truths = recn_gt['ground_truths']

client_recon_projected, client_gt_projected = dataset.decode_batch(reconstructions[0][0], standardized=True), dataset.decode_batch(ground_truths[0][0], standardized=True)


In [15]:
client_recon_projected.shape

(2000, 14)

In [16]:
idx1=10
client_gt_projected[:idx1], client_recon_projected[:idx1]
a,s,d,f,g,h=match_reconstruction_ground_truth(client_gt_projected[:idx1], client_recon_projected[:idx1], tolerance_map, return_indices=True)
a[:1]

array([['36.0', 'Private', '179464.0', 'HS-grad', '10.0',
        'Never-married', 'Adm-clerical', 'Own-child', 'White', 'Female',
        '-95.0', '21.0', '39.0', 'United-States']], dtype='<U32')

In [17]:
a.shape

(10, 14)

In [18]:
column_names = [
            'Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-Num', 'Marital-Status',
            'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-Gain', 'Capital-Loss',
            'Hours-per-week', 'Native-Country'
        ]
df = pd.DataFrame(client_recon_projected, columns=column_names)

# Convert numerical columns to integers where applicable
numeric_cols = ['Age', 'Fnlwgt', 'Education-Num', 'Capital-Gain', 'Capital-Loss', 'Hours-per-week']
df[numeric_cols] = df[numeric_cols].astype(float).astype(int)

df1=pd.read_csv(f'../clients_data/raw_data/client_0.data', delimiter=',',names=column_names, engine='python')
# label_col = df1.iloc[:, -1]

num_rows = len(df)
# print(df1)
# print(df1.iloc[:num_rows].values)

df['Income'] = df1.iloc[:num_rows, -1].values
df.head()
# df.to_csv("hello.data", index=False, header=False)

,Age,Workclass,Fnlwgt,Education,Education-Num,Marital-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-per-week,Native-Country,Income
0,36,Private,181261,HS-grad,10,Divorced,Transport-moving,Not-in-family,White,Male,38,77,39,United-States,<=50K
1,36,Private,179464,HS-grad,10,Never-married,Adm-clerical,Own-child,White,Female,-95,21,39,United-States,<=50K
2,38,Private,180127,11th,13,Never-married,Sales,Husband,White,Male,653,463,40,United-States,>50K
3,36,Private,172257,12th,10,Never-married,Other-service,Own-child,White,Female,19,19,39,United-States,<=50K
4,50,Private,249298,11th,11,Never-married,Adm-clerical,Own-child,White,Male,6741,755,39,United-States,<=50K


In [19]:
len(df)

2000

In [26]:
np.__version__

'1.22.4'

In [20]:
import numpy as np
recn_gt = f'../clients_data/reconstr_and_GT/inversion/normal/reconstructions_ground_truths_0.pkl'
dataset = f'../clients_data/reconstr_and_GT/inversion/normal/dataset_0.pkl'
tolerance_map = f'../clients_data/reconstr_and_GT/inversion/normal/tolerance_map_0.pkl'

with open(tolerance_map, 'rb') as file:
    tolerance_map = pickle.load(file)

with open(recn_gt, 'rb') as file:
    recn_gt = pickle.load(file)

with open(dataset, 'rb') as file:
    dataset = pickle.load(file)

reconstructions = recn_gt['reconstructions']
ground_truths = recn_gt['ground_truths']

client_recon_projected, client_gt_projected = dataset.decode_batch(reconstructions[0][0], standardized=True), dataset.decode_batch(ground_truths[0][0], standardized=True)


local_epochs=[5]
local_batch_sizes=[8]
epoch_prior_params=[0.01]
collected_data = np.zeros((len(local_epochs), len(local_batch_sizes), len(epoch_prior_params), 3, 5))
all_errors = []
cat_errors = []
cont_errors = []
idx=2000
client_gt_projected=client_gt_projected[:idx]
client_recon_projected_all_ctgan = np.load('all_chunks_ctgan.npy', allow_pickle=True)
client_recon_projected_all_tvae = np.load('all_chunks_tvae.npy', allow_pickle=True)



In [21]:
merged_data = np.concatenate([client_recon_projected_all_ctgan, client_recon_projected_all_tvae])

np.random.shuffle(merged_data)

In [22]:
merged_data.shape

(50, 2000, 14)

In [25]:
# merged_data[0].shape

In [ ]:
# client_recon_projected_all = merged_data
client_recon_projected_all=client_recon_projected_all_tvae

In [ ]:

batch_cost_all_collect=[]
batch_cost_cat_collect=[] 
batch_cost_cont_collect=[]
reorder_data = []

for i in range(len(client_recon_projected_all)):
    
    client_recon_projected= client_recon_projected_all[i]
    # print("client_recon_projected: ",client_recon_projected.shape)
    
    client_recon_projected=client_recon_projected[:idx]

    # _,batch_cost_all, batch_cost_cat, batch_cost_cont= match_reconstruction_ground_truth(client_gt_projected[:idx], client_recon_projected[:idx], tolerance_map, return_indices=True)
    a,s,d,f,g,h= match_reconstruction_ground_truth(client_gt_projected[:idx], client_recon_projected[:idx], tolerance_map, return_indices=True)
    reorder_data.append(a)
    batch_cost_all_collect.append(f)
    batch_cost_cat_collect.append(g)
    batch_cost_cont_collect.append(h)
    # if i ==0:
    #     break

for m in range(len(client_recon_projected_all)):
    for i, lepochs in enumerate(local_epochs):
        for j, lbatch_size in enumerate(local_batch_sizes):
            for k, epoch_prior_param in enumerate(epoch_prior_params):

                all_errors.append(np.mean(batch_cost_all_collect[m]))
                cat_errors.append(np.mean(batch_cost_cat_collect[m]))
                cont_errors.append(np.mean(batch_cost_cont_collect[m]))                        

                # np.mean(all_errors)
                collected_data[i, j, k, 0] = np.mean(all_errors), np.std(all_errors), np.median(all_errors), np.min(all_errors), np.max(all_errors)
                collected_data[i, j, k, 1] = np.mean(cat_errors), np.std(cat_errors), np.median(cat_errors), np.min(cat_errors), np.max(cat_errors)
                collected_data[i, j, k, 2] = np.mean(cont_errors), np.std(cont_errors), np.median(cont_errors), np.min(cont_errors), np.max(cont_errors)


    best_param_index = np.argmin(collected_data[i, j, :, 0, 0]).item()
    print(f'Performance at {lepochs} Epochs and {lbatch_size} Batch Size: {100*(1-collected_data[i, j, best_param_index, 0, 0]):.1f}% +- {100*collected_data[i, j, best_param_index, 0, 1]:.2f}')

    print(m)

Performance at 5 Epochs and 8 Batch Size: 58.1% +- 0.00
0
Performance at 5 Epochs and 8 Batch Size: 58.2% +- 0.11
1
Performance at 5 Epochs and 8 Batch Size: 58.2% +- 0.10
2
Performance at 5 Epochs and 8 Batch Size: 58.2% +- 0.11
3
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.15
4
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.14
5
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.18
6
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.18
7
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.17
8
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.16
9
Performance at 5 Epochs and 8 Batch Size: 58.4% +- 0.16
10
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.16
11
Performance at 5 Epochs and 8 Batch Size: 58.4% +- 0.15
12
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.15
13
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.15
14
Performance at 5 Epochs and 8 Batch Size: 58.3% +- 0.14
15
Performance at 5 Epochs and 8 Batch Size: 58.4% +- 0.15
16
Perform

In [ ]:
# 57%

In [ ]:
reorder_data[0][:3]

array([[36, 'Private', 172974, 'HS-grad', 10, 'Never-married',
        'Machine-op-inspct', 'Own-child', 'White', 'Male', 0, 19, 39,
        'United-States'],
       [91, 'Self-emp-inc', 172974, 'Assoc-acdm', 10,
        'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White',
        'Male', 0, 24, 88, 'United-States'],
       [50, 'Private', 172974, 'Prof-school', 15, 'Married-civ-spouse',
        'Prof-specialty', 'Husband', 'White', 'Male', 0, 24, 41,
        'United-States']], dtype=object)

In [ ]:
client_recon_projected_all[0][:3]

array([[38, 'Private', 179171, '9th', 10, 'Married-civ-spouse',
        'Machine-op-inspct', 'Unmarried', 'White', 'Female', 0, 19, 39,
        'United-States'],
       [56, 'Local-gov', 186614, 'Assoc-voc', 33, 'Widowed',
        'Exec-managerial', 'Other-relative', 'Other', 'Female', 1173,
        193, 43, 'Canada'],
       [36, 'Private', 174646, 'Prof-school', 10, 'Separated',
        'Other-service', 'Own-child', 'White', 'Male', 1103, 551, 55,
        'United-States']], dtype=object)

In [ ]:
num_elements = int(0.57 * 2000)
reorder_data[0][:num_elements]

array([[36, 'Private', 172208, ..., 23, 57, 'United-States'],
       [37, 'Never-worked', 172326, ..., 37, 39, 'United-States'],
       [56, 'State-gov', 173427, ..., 110, 86, 'United-States'],
       ...,
       [61, 'Private', 339821, ..., 2030, 100, 'Guatemala'],
       [37, 'Private', 172795, ..., 31, 39, 'United-States'],
       [38, 'Self-emp-inc', 175923, ..., 32, 39, 'United-States']],
      dtype=object)

In [ ]:
client_recon_projected_all[1][:num_elements]

array([[100, 'Private', 176532, ..., 898, 93, 'India'],
       [55, 'Private', 172285, ..., 24, 113, 'United-States'],
       [45, 'Private', 181062, ..., 936, 108, 'Taiwan'],
       ...,
       [62, 'Private', 182297, ..., 178, 113, 'Columbia'],
       [47, 'Private', 238742, ..., 903, 40,
        'Outlying-US(Guam-USVI-etc)'],
       [36, 'Private', 172917, ..., 41, 75, 'United-States']],
      dtype=object)

In [ ]:
reorder_data[0][:num_elements]

array([[82, 'Private', 172554, ..., 903, 61, 'United-States'],
       [42, 'Self-emp-inc', 240846, ..., 989, 96, 'China'],
       [57, 'Private', 181605, ..., 1078, 69, 'Cuba'],
       ...,
       [55, 'Private', 187616, ..., 40, 86, 'United-States'],
       [39, 'Never-worked', 175566, ..., 1006, 54, 'United-States'],
       [37, 'Private', 422393, ..., 1851, 48, 'United-States']],
      dtype=object)

In [ ]:
reorder_data[1][:num_elements].shape   

(840, 14)

In [56]:
combined_data = np.vstack([reorder_data[i][:num_elements] for i in range(25)])
combined_data.shape

(21000, 14)

In [58]:
combined_data

array([[82, 'Private', 172554, ..., 903, 61, 'United-States'],
       [42, 'Self-emp-inc', 240846, ..., 989, 96, 'China'],
       [57, 'Private', 181605, ..., 1078, 69, 'Cuba'],
       ...,
       [85, 'Private', 183244, ..., 859, 110, 'Columbia'],
       [37, 'Local-gov', 182528, ..., 298, 80, 'Germany'],
       [90, 'Private', 179206, ..., 762, 98, 'United-States']],
      dtype=object)

In [61]:
chunk_size = 2000
# arr = combined_data.iloc[:, :-1].to_numpy()
num_chunks = combined_data.shape[0] // chunk_size

chunk_list = np.split(combined_data[:num_chunks * chunk_size], num_chunks)


In [63]:
chunk_list[0].shape

(2000, 14)

In [65]:

batch_cost_all_collect=[]
batch_cost_cat_collect=[] 
batch_cost_cont_collect=[]
reorder_data = []
idx=2000
for i in range(len(chunk_list)):
    
    client_recon_projected= chunk_list[i]
    # print("client_recon_projected: ",client_recon_projected.shape)
    
    client_recon_projected=client_recon_projected[:idx]

    # _,batch_cost_all, batch_cost_cat, batch_cost_cont= match_reconstruction_ground_truth(client_gt_projected[:idx], client_recon_projected[:idx], tolerance_map, return_indices=True)
    a,s,d,f,g,h= match_reconstruction_ground_truth(client_gt_projected[:idx], client_recon_projected[:idx], tolerance_map, return_indices=True)
    reorder_data.append(a)
    batch_cost_all_collect.append(f)
    batch_cost_cat_collect.append(g)
    batch_cost_cont_collect.append(h)
    if i ==0:
        break

    
for m in range(len(client_recon_projected_all)):
    for i, lepochs in enumerate(local_epochs):
        for j, lbatch_size in enumerate(local_batch_sizes):
            for k, epoch_prior_param in enumerate(epoch_prior_params):

                all_errors.append(np.mean(batch_cost_all_collect[m]))
                cat_errors.append(np.mean(batch_cost_cat_collect[m]))
                cont_errors.append(np.mean(batch_cost_cont_collect[m]))                        

                # np.mean(all_errors)
                collected_data[i, j, k, 0] = np.mean(all_errors), np.std(all_errors), np.median(all_errors), np.min(all_errors), np.max(all_errors)
                collected_data[i, j, k, 1] = np.mean(cat_errors), np.std(cat_errors), np.median(cat_errors), np.min(cat_errors), np.max(cat_errors)
                collected_data[i, j, k, 2] = np.mean(cont_errors), np.std(cont_errors), np.median(cont_errors), np.min(cont_errors), np.max(cont_errors)


    best_param_index = np.argmin(collected_data[i, j, :, 0, 0]).item()
    print(f'Performance at {lepochs} Epochs and {lbatch_size} Batch Size: {100*(1-collected_data[i, j, best_param_index, 0, 0]):.1f}% +- {100*collected_data[i, j, best_param_index, 0, 1]:.2f}')

    print(m)

Performance at 5 Epochs and 8 Batch Size: 41.2% +- 2.52
0


IndexError: list index out of range